## **Gathering the Lending and Borrowing Rates**
**1. Fetching Symbols from Aave:** In this section, we retrieve the list of symbols available on the Aave platform. We use the Aave API to fetch the available symbols and store them in a list. We also check if the request was successful and handle any errors.

**2. Fetching Rates History for Symbols:** We define a function to fetch the average rates for a given reserve ID from the Aave API. This function sends a GET request with the reserve ID and retrieves the rates history data.

**3. Mapping Rates to Symbols:** We map the rates data to the corresponding symbols using the reserve ID for the Aave V2 market. We iterate through the reserves, create the reserve ID, and fetch the rates history for each symbol. The fetched rates data is stored in a dictionary.

**4. Extracting and Formatting Rates Data:** We extract the relevant data (liquidity rates, borrow rates, utilization rates, and stable borrow rates) from the fetched rates history and format it into a DataFrame. We also create timestamps for each data point and ensure the data is correctly structured.

**5. Fixing Date Format:** We fix any issues with the date format in the DataFrame. This involves extracting and converting the year, month, day, and hour components from the timestamps and creating a new datetime column.

**6. Cleaning and Preparing Data:** We clean the DataFrame by dropping unnecessary columns and ensuring the data is correctly formatted. We also handle any missing or zero values by replacing them with a placeholder value (-50).

**7. Filtering and Mapping Symbols:** We filter the symbols to include only those that are acceptable for our analysis. We also map the rates symbols to their corresponding price symbols from Binance.

**8. Sending Rates Data to the Database:** We establish a connection to the MySQL database and prepare the data for insertion. We create an insert statement and execute it to insert the rates data into the `crypto_lending_borrowing` table. Finally, we commit the transaction and close the database connection.

In [ ]:
import requests
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
import calendar
import os
import glob

In [ ]:
# Aave API endpoint for market data
url = "https://aave-api-v2.aave.com/data/markets-data"
# Send a GET request to the Aave API
response = requests.get(url)
symbols = []
# Check if the request was successful
if response.status_code == 200:
    count = 0
    res = response.json()
    for s in res['reserves']:
        symbols.append(s['symbol'])
else:
    print("Failed to retrieve data from Aave API")

print(len(symbols))
print(symbols)

237
['DAI', 'TUSD', 'USDC', 'USDT', 'SUSD', 'LEND', 'BAT', 'ETH', 'LINK', 'KNC', 'REP', 'MKR', 'MANA', 'ZRX', 'SNX', 'WBTC', 'BUSD', 'ENJ', 'REN', 'YFI', 'AAVE', 'UNI', 'AUSDT', 'AWBTC', 'AWETH', 'AYFI', 'AZRX', 'AUNI', 'AAAVE', 'ABAT', 'ABUSD', 'ADAI', 'AENJ', 'AKNC', 'ALINK', 'AMANA', 'AMKR', 'AREN', 'ASNX', 'ASUSD', 'ATUSD', 'AUSDC', 'ACRV', 'AGUSD', 'ABAL', 'AXSUSHI', 'ARENFIL', 'ARAI', 'AAMPL', 'AUSDP', 'ADPI', 'AFRAX', 'AFEI', 'ASTETH', 'AENS', 'AUST', 'ACVX', 'A1INCH', 'ALUSD', 'AAMMWETH', 'AAMMDAI', 'AAMMUSDC', 'AAMMUSDT', 'AAMMWBTC', 'AAMMUNIDAIWETH', 'AAMMUNIWBTCWETH', 'AAMMUNIAAVEWETH', 'AAMMUNIBATWETH', 'AAMMUNIDAIUSDC', 'AAMMUNICRVWETH', 'AAMMUNILINKWETH', 'AAMMUNIMKRWETH', 'AAMMUNIRENWETH', 'AAMMUNISNXWETH', 'AAMMUNIUNIWETH', 'AAMMUNIUSDCWETH', 'AAMMUNIWBTCUSDC', 'AAMMUNIYFIWETH', 'AAMMBPTWBTCWETH', 'AAMMBPTBALWETH', 'AAMMGUNIDAIUSDC', 'AAMMGUNIUSDCUSDT', 'AMDAI', 'AMUSDC', 'AMUSDT', 'AMWBTC', 'AMWETH', 'AMWMATIC', 'AMAAVE', 'AMGHST', 'AMBAL', 'AMDPI', 'AMCRV', 'AMSUSHI',

In [ ]:
# Aave API endpoint for rates history
rates_url = "https://aave-api-v2.aave.com/data/rates-history"

# Aave API endpoint for markets data
markets_url = "https://aave-api-v2.aave.com/data/markets-data"

# LendingPoolAddressesProvider for Aave V2 on Ethereum
lending_pool_provider_v2 = "0xb53c1a33016b2dc2ff3653530bff1848a515c8c5"

In [ ]:
# Function to fetch average rates for a given reserveId
def fetch_average_rates(reserveId):
    params = {
        "reserveId": reserveId,
        "from": 1580412380,  # 2020,1,30
        "resolutionInHours": 1
    }
    response = requests.get(rates_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve rates for {reserveId}")
        return None

In [ ]:
# Map the rates to symbols (using the reserveId for Aave V2 market)
rates_data = {}
for reserve in markets_data['reserves']:
    asset_address = reserve['underlyingAsset']  # assetAddress
    reserve_id = asset_address + lending_pool_provider_v2  # Create the reserveId
    symbol = reserve['symbol']  # Symbol for the asset
    rates_history = fetch_average_rates(reserve_id)

    if rates_history:
        rates_data[symbol] = rates_history


In [ ]:
symbols = []
liquidity_rates = []
borrow_rates = []
utilization_rates = []
stableborrow_rates = []
timestamps = []

# Extract data
for symbol, rates in rates_data.items():
  for rate in rates:
    symbols.append(symbol.split()[-1])  # Extract symbol from "Rates for SYMBOL"
    liquidity_rates.append(rate['liquidityRate_avg'])
    borrow_rates.append(rate['variableBorrowRate_avg'])
    utilization_rates.append(rate['utilizationRate_avg'])
    stableborrow_rates.append(rate['stableBorrowRate_avg'])

    # Create timestamp
    year = rate['x']['year']
    month = rate['x']['month']
    date = rate['x']['date']
    hours = rate['x']['hours']
    timestamp = f"{year}-{month:02d}-{date:02d} {hours:02d}:00:00"  # Format as YYYY-MM-DD HH:MM:SS
    timestamps.append(timestamp)

# Create DataFrame
df = pd.DataFrame({
    'Symbol': symbols,
    'Timestamp': timestamps,
    'liquidityRate_avg': liquidity_rates,
    'variableBorrowRate_avg': borrow_rates,
    'utilizationRate_avg' : utilization_rates,
    'stableBorrowRate_avg' : stableborrow_rates
})


In [ ]:
print(df['Timestamp'].min())
print(df['Timestamp'].max())

2020-10-30 18:00:00
2024-09-30 18:00:00


### Fixing the date format

In [ ]:
rates_df = df.copy()

In [ ]:
# Loop through each date, attempt conversion, and print non-convertible ones
rates_df['year'] = rates_df['Timestamp'].apply(lambda x:x[0:4])
rates_df['year'] = rates_df['year'].apply(lambda x: int(x))
# rates_df['year'].value_counts(dropna=False)
# it seems there is no problem with years

In [ ]:
rates_df['month'] = rates_df['Timestamp'].apply(lambda x:x[5:7])
# the problem might be here because of 00 , 01, ..
rates_df['month'] = rates_df['month'].apply(lambda x: int(x) + 1)
# rates_df['month'].value_counts(dropna=False)

In [ ]:
rates_df['day'] = rates_df['Timestamp'].apply(lambda x:x[8:10])
rates_df['day'] = rates_df['day'].apply(lambda x: int(x))
# print(len(rates_df['day'].value_counts(dropna=False)))
# rates_df['day'].value_counts(dropna=False)
# this also seems fine

In [ ]:
rates_df['hour'] = rates_df['Timestamp'].apply(lambda x:x[11:13])
rates_df['hour'] = rates_df['hour'].apply(lambda x: int(x))
# print(len(rates_df['hour'].value_counts(dropna=False)))
# rates_df['hour'].value_counts(dropna=False)
# this also seems fine

In [ ]:
rates_df['datetime'] = pd.to_datetime(rates_df[['year', 'month', 'day', 'hour']])
# The whole problem was because of the fact that months started from 0 instead of 1

In [ ]:
# testing the fact that month 9 should not have 31 days:
rates_df[(rates_df['month'] == 9) & (rates_df['day'] == 31)]

,Unnamed: 0,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg,year,month,day,hour,datetime


In [ ]:
# replacing
rates_df['Timestamp'] = rates_df['datetime']
rates_df.drop('datetime', inplace=True, axis=1)
print(rates_df['Timestamp'].isna().sum())

0


In [ ]:
# cleaning
rates_df.drop(['Unnamed: 0', 'year', 'month', 'day', 'hour'], inplace=True, axis=1)
print(rates_df.columns)
print(len(rates_df))
rates_df.head(3)

Index(['Symbol', 'Timestamp', 'liquidityRate_avg', 'variableBorrowRate_avg',
       'utilizationRate_avg', 'stableBorrowRate_avg'],
      dtype='object')
2141202


,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg
0,DAI,2020-12-01 16:00:00,0.000899,0.003996,0.083333,0.042083
1,DAI,2020-12-01 17:00:00,0.000000,0.000000,0.250000,0.046250
2,DAI,2020-12-01 18:00:00,0.000000,0.000000,0.250000,0.046250


In [ ]:
# checking type
type(rates_df['Timestamp'][1000])

pandas._libs.tslibs.timestamps.Timestamp

### Sending the rates to the database

In [ ]:
# MySQL database connection function
def connect_to_database():
    try:
        # Establishing connection to the database
        connection = mysql.connector.connect(
            host='crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com',
            database='Crypto',
            user='Jing', 
            password='Crypto12!'
        )

        if connection.is_connected():
            db_info = connection.get_server_info()
            print("Connected to MySQL database, MySQL Server version: ", db_info)
            return connection

    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

connection = connect_to_database()

Connected to MySQL database, MySQL Server version:  8.0.39


In [ ]:
df_rates = rates_df.copy()
print(len(df_rates))
df_rates.head(3)

2141202


,Unnamed: 0,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg
0,0,DAI,2020-12-01 16:00:00,0.000899,0.003996,0.083333,0.042083
1,1,DAI,2020-12-01 17:00:00,0.000000,0.000000,0.250000,0.046250
2,2,DAI,2020-12-01 18:00:00,0.000000,0.000000,0.250000,0.046250


In [ ]:
# Function to get all table names in the database
def get_table_names(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    return [table[0] for table in tables]

get_table_names(connection)

[bytearray(b'US_Bond_Yield'),
 bytearray(b'crypto_lending_borrowing'),
 bytearray(b'crypto_price'),
 bytearray(b'crypto_reference')]

In [ ]:
symbols = pd.DataFrame(columns = ['matching','rates','prices'])
rates_symbols = df_rates['Symbol'].unique()
symbols['rates'] = rates_symbols
symbols['matching'] = [x.lower() for x in rates_symbols]
symbols.head(3)

,matching,rates,prices
0,dai,DAI,NaN
1,tusd,TUSD,NaN
2,usdc,USDC,NaN


In [ ]:
path = 'prices/*.csv'
csv_files = glob.glob(path)
df_price = pd.concat([
    pd.read_csv(file).assign(symbol=os.path.splitext(os.path.basename(file))[0])
    for file in csv_files
], ignore_index=True)
print(df_price.columns)
print(len(df_price))


Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'close_time', 'quote_asset_volume', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'symbol'],
      dtype='object')
1187631


In [ ]:
prices_symbols = df_price['symbol'].unique()
len(prices_symbols)

38

In [ ]:
for price_symbol in prices_symbols:
     x = price_symbol.replace('USDT','')
     x = x.lower()
     if x in symbols['matching'].values:
          index = symbols[symbols['matching'] == x].index
          symbols.loc[index, 'prices'] = price_symbol
symbols.head(3)

,matching,rates,prices
0,dai,DAI,DAIUSDT
1,tusd,TUSD,TUSDUSDT
2,usdc,USDC,USDCUSDT


In [ ]:
accpetable_tokens = ['bat', 'link', 'knc', 'mkr', 'mana', 'zrx', 'snx', 'wbtc', 'enj', 'ren', 'yfi', 'uni', 'crv', 'bal', 'ens', '1inch']
len(accpetable_tokens)

16

In [ ]:
symbols = symbols[symbols['matching'].isin(accpetable_tokens)]
print(len(symbols))
symbols.head(3)

16


,matching,rates,prices
5,bat,BAT,BATUSDT
6,link,LINK,LINKUSDT
7,knc,KNC,KNCUSDT


In [ ]:
df_rates_filtered = df_rates[df_rates['Symbol'].isin(symbols['rates'].to_list())]
df_rates_filtered.reset_index(inplace=True, drop=True)
print(len(df_rates_filtered))

515214


In [ ]:
def replace_symbol(symbol):
     return symbols[symbols['rates'] == symbol]['prices'].values[0]
print(replace_symbol('BAT'))

BATUSDT


In [ ]:
df_rates_filtered['Symbol'] = df_rates_filtered['Symbol'].apply(replace_symbol)
# df_rates_filtered['Symbol'].value_counts(dropna=False)

In [ ]:
# we change the zero and NA's to -50
df_rates_filtered.replace(0, -50, inplace=True)
count_minus_50 = (df_rates_filtered == -50).sum().sum()
print(count_minus_50)
print(df_rates_filtered.isna().sum())

215011
Unnamed: 0                0
Symbol                    0
Timestamp                 0
liquidityRate_avg         0
variableBorrowRate_avg    0
utilizationRate_avg       0
stableBorrowRate_avg      0
dtype: int64


C:\Users\hgnik\AppData\Local\Temp\ipykernel_43048\1387197028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rates_filtered.replace(0, -50, inplace=True)


In [ ]:
# Convert the 'timestamp' column to datetime format, if it's not already
df_rates_filtered.reset_index(inplace=True, drop=True)
df_rates_filtered['date'] = pd.to_datetime(df_rates_filtered['Timestamp'])

# Format the datetime to MySQL's expected string format
df_rates_filtered['date'] = df_rates_filtered['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
cursor = connection.cursor()
unique_symbols = df_rates_filtered['Symbol'].unique()

In [ ]:
insert_statement = """
    INSERT IGNORE INTO crypto_lending_borrowing (crypto_symbol, date, lending_rate, borrowing_rate, utilization_rate, stable_borrow_rate)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

data = df_rates_filtered[['Symbol', 'date', 'liquidityRate_avg', 'variableBorrowRate_avg', 'utilizationRate_avg', 'stableBorrowRate_avg']].values.tolist()
cursor.executemany(insert_statement, data)

# Commit and close connection
connection.commit()
cursor.close()
connection.close()

In [ ]:
connection = connect_to_database()
# Function to view data from a specific table
def view_table_data(connection, table_name, limit=300):
    cursor = connection.cursor()
    query = f"SELECT * FROM {table_name} LIMIT %s;"
    cursor.execute(query, (limit,))
    data = cursor.fetchall()
    return data

view_table_data(connection,'crypto_lending_borrowing', limit=30)

Connected to MySQL database, MySQL Server version:  8.0.39


[(1,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 14, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.030000')),
 (2,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 15, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.030000')),
 (3,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 16, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.013598'),
  Decimal('0.030000')),
 (4,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 17, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.014834'),
  Decimal('0.030000')),
 (5,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 18, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.014834'),
  Decimal('0.030000')),
 (6,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 19, 0),
  Decimal('-50.000000'),
  Decimal('-50.000000'),
  Decimal('0.014834'),
  Decimal('0.030000')),
 (7,
  'BATUSDT',
  datetime.datetime(2020, 12, 2, 20, 0),
 

### End of Notebook